# mcmc

> Fill in a module description here

In [ ]:
#| default_exp mcmc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def ln_prior(): pass

def ln_like(): pass

def ln_prob(): pass

def burn_in(): pass

def mcmc_run(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()